# ASL Recognizer Model Notebook | Group 03 - NT536.O11.MMCL

## Prerequisites

Install the MediaPipe Model Maker package.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install --upgrade pip
!pip install mediapipe-model-maker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

Import the required libraries.

In [3]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


## Data Loading

This end-to-end example uses Model Maker to customize a model for on-device gesture recognition.

### Get the dataset from Kaggle

In [8]:
!pip install kaggle

In [9]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [12]:
!kaggle datasets download -d prathumarikeri/american-sign-language-09az

100% 4.18G/4.20G [00:52<00:00, 116MB/s]
100% 4.20G/4.20G [00:53<00:00, 85.0MB/s]


In [13]:
!unzip /content/american-sign-language-09az.zip

Streaming output truncated to the last 5000 lines.
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2863).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2864).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2865).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2866).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2867).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2868).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2869).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (287).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2870).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2871).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2872).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2873).jpg  
  inflating: A

In [14]:
dataset_path = "/content/American"

### Reduce each folder to only 500 images

In [15]:
import os

# Prompt for the directory path
directory_path = dataset_path
# Set the desired file count per folder
max_files_per_folder = 500

# Check if the directory exists
if not os.path.isdir(directory_path):
    print(f"Error: Directory '{directory_path}' does not exist.")
    exit()

# Get a list of all folders in the specified directory
folders = [f for f in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, f))]

for folder in folders:
    # Get the full path to the current folder
    current_folder_path = os.path.join(directory_path, folder)

    # Get a list of files in the current folder
    files = os.listdir(current_folder_path)

    # Sort files by most recently modified (newest first)
    files.sort(key=lambda f: os.path.getmtime(os.path.join(current_folder_path, f)), reverse=True)

    # Iterate through files and delete if count exceeds limit
    for i in range(max_files_per_folder, len(files)):
        os.remove(os.path.join(current_folder_path, files[i]))

print(f"Finished processing folders in '{directory_path}'.")


Finished processing folders in '/content/American'.


### Check labels

In [16]:
print(dataset_path)
labels = []
for i in os.listdir(dataset_path):
  if os.path.isdir(os.path.join(dataset_path, i)):
    labels.append(i)
print(labels)

/content/American
['d', '0', 'e', 'w', 'm', '2', '8', 'n', '9', 'z', '7', 'b', 'k', 't', 'a', 'g', '5', '4', 'v', '3', '6', 'p', '1', 's', 'none', 'h', 'o', 'u', 'y', 'x', 'l', 'f', 'q', '.ipynb_checkpoints', 'i', 'j', 'r', 'c']


In [18]:
print(len(labels))

38


In [ ]:
import pandas as pd

activity_dataframes = []

# Iterate through subfolders
for activity_folder in os.listdir(dataset_path):
    activity_folder_path = os.path.join(dataset_path, activity_folder)

    # Check if it's a directory
    if os.path.isdir(activity_folder_path):
        # Initialize an empty list to store DataFrames for each file in the subfolder

        # Iterate through files in the subfolder
        for file_name in os.listdir(activity_folder_path):
                file_path = os.path.join(activity_folder_path, file_name)

                # Read the data from each CSV file into a DataFrame
                df = pd.DataFrame({'ImagePath':[file_path]})
                df['label'] = activity_folder
                activity_dataframes.append(df)

In [ ]:
result_df = pd.concat(activity_dataframes, ignore_index=True)

In [ ]:
# Assuming your DataFrame is called "df"
label_counts = result_df['label'].value_counts()
print(label_counts)

d       1570
0       1570
6       1570
p       1570
1       1570
s       1570
h       1570
o       1570
u       1570
y       1570
x       1570
l       1570
f       1570
q       1570
i       1570
j       1570
r       1570
3       1570
v       1570
4       1570
5       1570
e       1570
w       1570
m       1570
2       1570
8       1570
n       1570
9       1570
z       1570
7       1570
b       1570
k       1570
t       1570
a       1570
g       1570
c       1570
none    1500
Name: label, dtype: int64


## Model Training
The workflow consists of 4 steps which have been separated into their own code blocks.

**Preprocess the dataset**

Load the dataset located at `dataset_path` by using the `Dataset.from_folder` method. When loading the dataset, run the pre-packaged hand detection model from MediaPipe Hands to detect the hand landmarks from the images. Any images without detected hands are ommitted from the dataset. The resulting dataset will contain the extracted hand landmark positions from each image, rather than images themselves.

The `HandDataPreprocessingParams` class contains two configurable options for the data loading process:
* `shuffle`: A boolean controlling whether to shuffle the dataset. Defaults to true.
* `min_detection_confidence`: A float between 0 and 1 controlling the confidence threshold for hand detection.

Split the dataset: 80% for training, 10% for validation, and 10% for testing.

In [19]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=dataset_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

**Train the model**

Train the custom gesture recognizer by using the create method and passing in the training data, validation data, model options, and hyperparameters. For more information on model options and hyperparameters, see the [Hyperparameters](#hyperparameters) section below.

In [21]:
hparams = gesture_recognizer.HParams(learning_rate=0.001, export_dir="exported_model_2",epochs=50,batch_size=10)
model_options = gesture_recognizer.ModelOptions(dropout_rate=0.5)
options = gesture_recognizer.GestureRecognizerOptions(model_options=model_options, hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization_1 (Bat  (None, 128)               512       
 chNormalization)                                                
                                                                 
 re_lu_1 (ReLU)              (None, 128)               0         
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 38)                4902      
 out (Dense)                                                     
                                                           

**Evaluate the model performance**

After training the model, evaluate it on a test dataset and print the loss and accuracy metrics.

In [22]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

1078/1078 [==============================] - 6s 2ms/step - loss: 0.0616 - categorical_accuracy: 0.9731
Test loss:0.06162991002202034, Test accuracy:0.9730983376502991


**Export to Tensorflow Lite Model**

After creating the model, convert and export it to a Tensorflow Lite model format for later use on an on-device application. The export also includes model metadata, which includes the label file.

In [29]:
model.export_model()
!ls exported_model_2

Using existing files at /tmp/model_maker/gesture_recognizer/gesture_embedder.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/canned_gesture_classifier.tflite
asl_model.tflite			checkpoint		 logs
best_model_weights.data-00000-of-00001	epoch_models		 metadata.json
best_model_weights.index		gesture_recognizer.task
